# Semi-Supervised Concept Drift Algorithms

In [23]:
import pandas as pd
import numpy as np
import math
from sklearn import svm

import warnings
warnings.filterwarnings("ignore")

# Read Data

In [2]:
# Subject 1

# Defining Filepaths
filepath = 'test_dataset'

# Creating a list of values for Subject 1 by randomly combining the different file
num_iter = 50
combined_df_sub_1 = pd.DataFrame()

curr_file_1 = filepath + '/sub_1_combined_data.csv'
curr_file_2 = filepath + '/sub_2_combined_data.csv'

curr_df_1 = pd.read_csv(curr_file_1)
curr_df_2 = pd.read_csv(curr_file_2)


curr_df_1.drop('Unnamed: 0.1',axis=1,inplace=True)
curr_df_2.drop('Unnamed: 0.1',axis=1,inplace=True)

curr_df_1.drop('Unnamed: 0',axis=1,inplace=True)
curr_df_2.drop('Unnamed: 0',axis=1,inplace=True)

curr_df_1.drop('outlier',axis=1,inplace=True)
curr_df_2.drop('outlier',axis=1,inplace=True)

curr_df_1.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type
0,1.528132,-0.733896,0.696372,0.741895,0.669768,-0.031672,0.316738,0.778180,1.082764,0.294894,-0.184493,0.377542,dws
1,1.527992,-0.716987,0.677762,0.753099,0.657116,-0.032255,0.842032,0.424446,0.643574,0.219405,0.035846,0.114866,dws
2,1.527765,-0.706999,0.670951,0.759611,0.649555,-0.032707,-0.138143,-0.040741,0.343563,0.010714,0.134701,-0.167808,dws
3,1.516768,-0.704678,0.675735,0.760709,0.647788,-0.041140,-0.025005,-1.048717,0.035860,-0.008389,0.136788,0.094958,dws
4,1.493941,-0.703918,0.672994,0.760062,0.647210,-0.058530,0.114253,-0.912890,0.047341,0.199441,0.353996,-0.044299,dws


In [3]:
curr_df_1 = curr_df_1.rename(columns={"Unnamed: 0":"ind"})

In [4]:
curr_df_1["type"].value_counts()

sit    20584
std    12747
wlk    11112
jog     6428
ups     6336
dws     5105
Name: type, dtype: int64

In [5]:
curr_df_1.shape

(62312, 13)

In [6]:
curr_df_1.dtypes

attitude.roll         float64
attitude.pitch        float64
attitude.yaw          float64
gravity.x             float64
gravity.y             float64
gravity.z             float64
rotationRate.x        float64
rotationRate.y        float64
rotationRate.z        float64
userAcceleration.x    float64
userAcceleration.y    float64
userAcceleration.z    float64
type                   object
dtype: object

# Create Labels in Dataset

# sudden1

In [7]:
df_test=pd.read_csv('yifei_dataset_4.27_sudden30high200.csv')
df_test.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,6,0,0,0,0,0,6
1,1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,3,0,0,3,0,0,0
2,2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,1,0,0,0,0,0
3,3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,6,0,0,0,0,0,6
4,4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,6,0,0,0,0,0,6


In [8]:
df_test["type"]

0        6
1        3
2        1
3        6
4        6
        ..
62307    4
62308    5
62309    1
62310    4
62311    4
Name: type, Length: 62312, dtype: int64

In [9]:
df_test1 = df_test.copy()
df_test1.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X = df_test1.to_numpy()

In [10]:
df_test.type.value_counts()

3    19644
4    12518
5    11038
2     6726
6     5773
1     5616
0      997
Name: type, dtype: int64

# No Drift

In [11]:
df11=pd.read_csv('SampleDriftInjection/NoDrift.csv')
df11.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type
0,3675,0.025033,-1.202555,1.409351,0.009010,0.932962,-0.359863,0.831373,-3.207910,1.773906,-0.087593,-0.088144,0.401702,1
1,47982,0.215725,-0.475396,-0.098294,0.190319,0.457691,-0.868503,-0.000831,0.009239,0.012150,0.001148,0.007473,-0.008587,3
2,32019,-1.882635,-1.172086,-2.556060,-0.369506,0.921562,0.119113,-0.963449,0.887903,0.567634,-0.449860,-0.484764,0.024732,5
3,4865,-1.488771,-1.300567,0.896711,-0.266055,0.963710,-0.021872,0.147023,0.327224,0.252847,0.155169,-0.258983,0.102073,2
4,25950,0.853153,-1.323178,0.511841,0.184644,0.969499,-0.161178,-0.083820,-0.338874,-0.219943,-0.064878,-0.024965,0.071929,4


In [12]:
df_test11 = df11.copy()
df_test11.drop(columns=["type","Unnamed: 0"], inplace=True)
test_X1 = df_test11.to_numpy()

# Original

In [13]:
# create labels
dictionary = {"sit":0, "std":1, "wlk":2, "jog":3, "ups":4, "dws":5}
dictionary

{'sit': 0, 'std': 1, 'wlk': 2, 'jog': 3, 'ups': 4, 'dws': 5}

In [8]:
curr_df_1["encoded"] = curr_df_1["type"].apply(lambda x: dictionary[x])

In [9]:
# shuffle the DataFrame rows
curr_df_11 = curr_df_1.sample(frac = 1, random_state=10)

In [10]:
curr_df_11[:600]["encoded"].value_counts()

0    202
1    119
2    105
4     61
5     57
3     56
Name: encoded, dtype: int64

In [11]:
curr_df_11.drop(columns=["type"], inplace=True)
curr_df_11.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,encoded
34498,-3.119799,-0.435769,2.808867,-0.019756,0.422108,0.906330,-0.839569,-2.625599,-0.746496,0.357265,0.221188,-0.927754,2
17136,0.319598,-0.385717,0.047983,0.291102,0.376223,-0.879611,0.043646,-0.008660,-0.031626,0.002569,-0.000994,-0.016171,0
41849,-2.843994,-0.473762,3.103179,-0.260929,0.456237,0.850743,1.498496,0.761328,-0.171520,0.404773,0.104981,-1.186223,3
9595,0.356689,-0.442954,-0.102534,0.315475,0.428610,-0.846622,-0.025042,-0.034777,-0.094163,-0.001556,0.008524,-0.011319,0
35280,2.569419,-0.996903,2.479837,0.293962,0.839793,0.456435,1.360581,-1.830629,0.778301,-0.029421,1.071461,-1.307342,2


# Train base classifier

In [14]:
df_test.head()

,Unnamed: 0,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,6,0,0,0,0,0,6
1,1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,3,0,0,3,0,0,0
2,2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,1,0,0,0,0,0
3,3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,6,0,0,0,0,0,6
4,4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,6,0,0,0,0,0,6


In [15]:
df_test1.head()

,attitude.roll,attitude.pitch,attitude.yaw,gravity.x,gravity.y,gravity.z,rotationRate.x,rotationRate.y,rotationRate.z,userAcceleration.x,userAcceleration.y,userAcceleration.z,type1_drift,type2_drift,type3_drift,type4_drift,type5_drift,type6_drift
0,-2.562144,-1.219983,-3.050600,-0.188176,0.939093,0.287564,-0.482461,-0.225021,-0.622184,0.184102,-0.178611,-0.341397,0,0,0,0,0,6
1,0.341979,-0.367058,0.116501,0.313013,0.358871,-0.879337,-0.015481,-0.017824,0.007785,0.002478,-0.002655,-0.013454,0,0,3,0,0,0
2,-2.104340,-1.225199,2.236723,-0.291675,0.940873,0.172289,0.320235,-0.378826,-0.375082,0.371951,0.409285,-0.202959,1,0,0,0,0,0
3,-2.749196,-0.528337,2.947486,-0.330262,0.504097,0.798006,4.186034,-6.205776,-2.193666,1.084580,2.214027,-2.280946,0,0,0,0,0,6
4,1.515196,-0.736980,0.985010,0.739357,0.672055,-0.041151,-2.032415,8.415040,-1.795768,-0.728463,-0.730648,1.059584,0,0,0,0,0,6


In [16]:
def base_train(training, train_label, kernel_type="poly"):
    base_classifier = svm.SVC(kernel=kernel_type)
    base_classifier.probability=True
    base_classifier.fit(training, train_label)
    return base_classifier

# Manually Add Drift

In [1]:
# 99% of the rest of the data
others = curr_df_11[600:]
others.reset_index(inplace=True)
#label for the training set
other_label = others["encoded"]
others.drop(columns=["encoded"], inplace=True)
others.drop(columns=["index"], inplace=True)

In [15]:
others.shape

(61712, 12)

In [16]:
#add drift: by flipping sign of numerical features, 1000 chunks by 1000 chunks
drifts = range(0,61712,1000)
drifts = list(drifts)
len(drifts)

62

In [17]:
#index for unflipped rows
unflipped = []
for i in range(0,len(drifts),2):
    unflipped.extend(list(range(drifts[i],drifts[i+1])))
len(unflipped)

31000

In [18]:
#add drift: by flipping sign of numerical features, 1000 chunks by 1000 chunks
for index, row in others.iterrows():
    if index not in unflipped:
        others.iloc[index,:] = others.iloc[index,:].apply(lambda x: -x)

In [19]:
X = others.to_numpy()

# Apply Drift Detection Algorithm

## Fixed Reference Window

In [19]:
def FRW(X, n, base_classifier, retrain=False, num_std=1, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # move sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #calculate window stats
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            #determine if drift happens
            if m_cur >= m_ref + num_std*(s_ref/sqrt_n):
                lst.append(idx-n)
                #if to retrain the base classifier
                if retrain:
                    #learn a new base classifier
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])
                #initialize window as empty
                cur_idx = []
                ref = []
                cur = []
    return lst, len(lst)

## Moving Reference Window

In [20]:
def MRW(X, n, base_classifier, retrain=False, num_std=2, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # move sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window 
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            if m_cur >= m_ref + num_std*(s_ref/sqrt_n):
                lst.append(idx-n)
                #learn a new base classifier
                if retrain:
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])
                #initialize window as empty
                cur_idx = []
                ref = []
                cur = [] 
    return lst, len(lst)

## Ensemble Reference Window

In [21]:
def ERW(X, n, base_classifier, retrain=False, num_std=4, train_labels=None):    
    #window size n
    sqrt_n = math.sqrt(n)
    c4 = math.sqrt(2/(n-1))*math.factorial(int(n/2) - 1)/math.factorial(int((n-1)/2) - 1)
    
    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    m_lst = []
    s_lst = []

    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)
        # ensemble sliding window
        if len(ref) < n:
            ref.append(distance)
        elif len(cur) < n:
            cur.append(distance)
            cur_idx.append(idx)
        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window lists
            m_lst.append(np.mean(cur))
            s_lst.append(np.mean(cur)) 
            #calculate reference window mean
            m_ref = np.mean(ref)
            #determine if drift happens
            if np.mean(m_lst) + num_std*(np.std(m_lst)/sqrt_n)*c4 <= m_ref:
                lst.append(idx-n)
                if retrain:
                    #learn a new base classifier
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])
                #initialize window as empty
                ref = []
                cur = []
                cur_idx = []
                m_lst = []
                s_lst = []   
    
    return lst, len(lst)

## Dynamically updating window size
- How to introduce metrics that determines the window size
    - if only 1 drifts detected in 2 window lengths: make the window size smaller
    - if half of the drifts are detected within one window, then increase the window size

In [57]:
def DMRW(X, base_classifier, retrain=False, num_std=2, train_labels=None):    
    #window size n
    min_window_size = 100
    max_window_size = 400
    cur_window_size = min_window_size
    ref_window_size = min_window_size
    cur_window_size = min_window_size

    #create sliding windows
    ref = []
    cur = []
    cur_idx = []
    idx = -1
    lst = []
    for i in X:
        idx += 1
        # calculate x
        prob = base_classifier.predict_proba([i])
        y_index = np.argmax(prob)
        y = [0]*6
        y[y_index] = 1
        distance = np.sum((np.array(y) - np.array(prob))**2).round(4)

        # move sliding windows
        if len(ref) < cur_window_size:
            ref.append(distance)
        elif len(cur) < cur_window_size:
            cur.append(distance)
            cur_idx.append(idx)

        #dynamically updating the window size
        elif len(lst)>2 and lst[-2]-lst[-1] > 2*cur_window_size:
            cur_window_size = max(min_window_size,cur_window_size-min_window_size)
        elif len(lst)>12 and lst[len(lst)//2]-lst[-1] < cur_window_size:
            cur_window_size = min(max_window_size,cur_window_size+min_window_size)

        else:
            #move reference window by 1
            ref.pop(0)
            ref.append(cur[0])
            #move current window by 1
            cur.pop(0)
            cur.append(distance)
            cur_idx.pop(0)
            cur_idx.append(idx)
            #append to current window 
            m_cur = np.mean(cur)
            m_ref = np.mean(ref)
            s_ref = np.std(ref)
            if m_cur >= m_ref + num_std*(s_ref/math.sqrt(cur_window_size)):
                lst.append(idx-cur_window_size)

                #learn a new base classifier
                if retrain:
                    base_classifier.probability=True
                    base_classifier.fit(X[cur_idx], train_labels[cur_idx])

                #initialize window as empty
                cur_idx = []
                ref = []
                cur = [] 
    return lst, len(lst)

In [58]:
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])
DMRW_lst_f, DMRW_length_f = DMRW(test_X, base_classifier,retrain=False, num_std=1)

In [59]:
DMRW_lst_f

[239, 456, 739, 940, 1198, 1640, 1841, 2165, 2366, 2647, 2904, 3105, 3306]

In [60]:
DMRW_length_f

13

In [32]:
MRW_lst_f,MRW_length_f = MRW(test_X, 100, base_classifier,False, num_std)

In [33]:
MRW_lst_f

[239,
 456,
 739,
 940,
 1198,
 1640,
 1841,
 2165,
 2366,
 2647,
 2905,
 3247,
 3485,
 3691,
 4056,
 4305,
 4590,
 4791,
 5033,
 5346,
 5640,
 5841,
 6129,
 6330,
 6644,
 6892,
 7261,
 7462,
 7663,
 8098,
 8445,
 8717,
 8918,
 9149,
 9363,
 9564,
 9765,
 9966,
 10260,
 10687,
 10905,
 11106,
 11307,
 11508,
 11709,
 11910,
 12117,
 12347,
 12548,
 12809,
 13087,
 13288,
 13490,
 13691,
 13892,
 14206,
 14407,
 14638,
 14839,
 15082,
 15461,
 15809,
 16040,
 16241,
 16447,
 16648,
 16851,
 17052,
 17253,
 17547,
 17822,
 18023,
 18224,
 18462,
 18752,
 19012,
 19447,
 19648,
 19895,
 20096,
 20307,
 20509,
 20710,
 20911,
 21112,
 21405,
 21619,
 21820,
 22021,
 22222,
 22445,
 22646,
 22847,
 23048,
 23249,
 23515,
 23718,
 23939,
 24140,
 24341,
 24542,
 24743,
 24980,
 25301,
 25502,
 25726,
 25961,
 26225,
 26489,
 26709,
 26923,
 27124,
 27492,
 27752,
 28089,
 28413,
 28614,
 28815,
 29016,
 29262,
 29463,
 29677,
 29878,
 30127,
 30370,
 30594,
 30818,
 31067,
 31451,
 31764,
 3

In [34]:
MRW_length_f

243

## Test Performance

In [2]:
#test combinations of window size
n=200
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

for num_std in [1,2,3,4]:
    print(num_std)
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    

In [119]:
ERW_length_f_lst

[119, 93, 46, 21]

In [117]:
FRW_length_f_lst

[37, 24, 16, 17]

In [118]:
MRW_length_f_lst

[124, 121, 115, 109]

In [131]:
print(FRW_lst_f_lst[1])

[200, 1340, 4140, 14782, 30987, 33191, 34138, 40492, 41386, 42558, 43450, 45677, 46548, 49834, 50711, 52930, 53816, 54988, 55972, 57037, 58064, 59077, 60185, 61127]


In [122]:
print(ERW_lst_f_lst[1])

[1479, 2012, 2413, 2814, 3215, 3907, 4308, 5552, 6180, 6581, 7261, 7662, 9049, 9459, 11200, 11612, 12013, 12414, 12815, 13903, 14982, 15383, 15784, 16302, 16733, 17134, 17535, 18435, 18952, 19353, 19754, 20155, 20721, 21280, 21681, 22082, 25607, 26011, 26610, 27044, 27445, 27846, 28334, 28844, 29245, 29646, 30047, 31286, 31687, 32380, 33393, 34438, 35474, 36499, 37525, 37926, 38617, 39074, 39643, 40664, 41198, 41688, 42242, 42689, 43774, 44275, 44828, 45294, 45877, 46890, 47291, 47960, 48471, 48937, 49338, 50034, 51034, 51435, 53094, 54151, 55184, 55585, 56256, 56723, 57294, 58295, 58815, 59336, 59828, 60391, 60930, 61395, 62067]


In [124]:
len(ERW_lst_f_lst[1])

93

In [ ]:
#test combinations of window size
n_lst = np.arange(100,1000,100)

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []

for n in n_lst:
    print(n)
    
    FRW_lst_t,FRW_length_t = FRW(X, n, True)
    FRW_lst_f,FRW_length_f = FRW(X, n, False)
    MRW_lst_t,MRW_length_t = MRW(X, n, True)
    MRW_lst_f,MRW_length_f = MRW(X, n, False)
    
    FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    
    FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)

100
200
300
400
500
600
700
800


In [133]:
#test combinations of window size
n=200
#define base_classifier
base_classifier = base_train(df_test1[:1000], df_test["type"][:1000])

FRW_lst_t_lst = []
FRW_lst_f_lst = []
MRW_lst_t_lst = []
MRW_lst_f_lst = []
ERW_lst_t_lst = []
ERW_lst_f_lst = []

FRW_length_t_lst = []
FRW_length_f_lst = []
MRW_length_t_lst = []
MRW_length_f_lst = []
ERW_length_t_lst = []
ERW_length_f_lst = []

for num_std in [5,6,7]:
    print(num_std)
    
    #FRW_lst_t,FRW_length_t = FRW(test_X, n, base_classifier,True, num_std)
    FRW_lst_f,FRW_length_f = FRW(test_X, n, base_classifier,False, num_std)
    #MRW_lst_t,MRW_length_t = MRW(test_X, n, base_classifier,True, num_std)
    MRW_lst_f,MRW_length_f = MRW(test_X, n, base_classifier,False, num_std)
    #ERW_lst_t,ERW_length_t = ERW(test_X, n, base_classifier,True, num_std)
    ERW_lst_f,ERW_length_f = ERW(test_X, n, base_classifier,False, num_std)
    
    #FRW_lst_t_lst.append(FRW_lst_t)
    FRW_lst_f_lst.append(FRW_lst_f)
    #MRW_lst_t_lst.append(MRW_lst_t)
    MRW_lst_f_lst.append(MRW_lst_f)
    #ERW_lst_t_lst.append(ERW_lst_t)
    ERW_lst_f_lst.append(ERW_lst_f)
    
    #FRW_length_t_lst.append(FRW_length_t)
    FRW_length_f_lst.append(FRW_length_f)
    #MRW_length_t_lst.append(MRW_length_t)
    MRW_length_f_lst.append(MRW_length_f)
    #ERW_length_t_lst.append(ERW_length_t)
    ERW_length_f_lst.append(ERW_length_f)
    

5
6
7


## Comments on Algorithm results

- Algorithm Overview
    - When the access to the labels is limited, it utilizes the probability estimates for class prediction in order to describe the behavior patterns of classifier.
    - Three approaches based on two types of windows, a reference window and a detection window, are pro- posed.
    - For the Dynamically updating window size approach, adaptive thresholds for numbers of std and window sizes based on number of drifts detectd in the current window size are developed.

- Algorithmic Experiments:
    - On the Motion Sense Dataset, if we define drift as greater than 1 standard deviation of the reference window:
        - Using Fixed Reference window, then we could detect 6/61 drifts in the concept drift, that is about 10% of our manually created drifts.
        - Using Moving Reference Window, then we could detect 46/61 drifts in the concept drift, that is about 75% of our manually created drifts.
        - Using Ensemble Reference Window, then we could detect 30/61 drifts in the concept drift, that is about 50% of our manually created drifts.
        - Larger Standard Deviations, less false positives but also less drifts.

- Future Directions:
    - **Domain Adaptation**: Study methods for domain adaptation in the context of concept drift. Domain adaptation techniques aim to transfer knowledge from a source domain to a target domain, allowing models to adapt to changing data distributions and concepts.

    - **Feature Selection and Extraction**: Investigate feature selection or extraction techniques that are robust to concept drift. By focusing on the most informative and relevant features, models can be more resilient to changes in the data distribution.

    - **Anomaly Detection**: Explore the intersection of concept drift and anomaly detection. Anomaly detection methods can be used to identify abnormal patterns or instances that indicate concept drift, enabling proactive adaptation of models.

    - **Interpretability and Explainability**: Enhance the interpretability and explainability of concept drift algorithms. Developing methods to explain when and why concept drift occurs can facilitate trust and understanding in the decision-making process.

    - **Real-Time Visualization and Monitoring**: Create real-time visualization and monitoring tools that provide interactive visualizations of concept drift detection results. These tools can help practitioners track concept drift over time and make timely adjustments to models.
